In [1]:
import pandas as pd
import requests
import json
from datetime import datetime
import sqlalchemy
from sqlalchemy import create_engine
from google.cloud import bigquery
import os
import pyarrow 

In [2]:
#To find the Service Account Key location in this project
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "D:\\Python\\Financial_data_pipeline\\exchange-rate-pipeline-476115-5100605ea811.json"

In [3]:
#API token for exchangerate.host
api_token = "8ed177bf655d362a495251f500f37a35"

list_url = f"https://api.exchangerate.host/list?access_key={api_token}"

In [4]:
#to save the currency list as json
# response = requests.get(list_url)

# currency_list = response.json()

# with open("currency_list.json", "w") as f:
#     json.dump(currency_list, f)

In [5]:
#to get the current date
# today = datetime.today().strftime("%Y-%m-%d")

In [6]:
def extract_curr(base_currency="IDR"):
  # url = f"https://api.exchangerate.host/timeframe?access_key={api_token}&source={base_currency}&start_date=2025-09-01&end_date={today}"
  url = f"https://api.exchangerate.host/timeframe?access_key={api_token}&source={base_currency}&start_date=2025-09-01&end_date=2025-10-24"
  response = requests.get(url)

  data = response.json()

  #save the value as JSON
  with open("historical_curr.json", "w") as f:
    json.dump(data, f)

  return data

In [7]:
def transform_curr(data):
  historical_curr = data['quotes']
  records = []
  #opening the dictionary
  for date, rates in historical_curr.items():
    for currency, rate in rates.items():
      records.append({
          "Target_Currency" : currency,
          "Base_Currency" : "IDR",
          "Date" : date,
          "Exchange_Rate" : rate
      })
  df = pd.DataFrame(records)

  df["Rate_in_IDR"] = 1 / df["Exchange_Rate"]
  df["Target_Currency"] = df['Target_Currency'].str.replace('IDR', '')

  # Reload the currency DataFrame from the JSON file
  with open("currency_list.json","r") as file:
    curr = json.load(file)
  currency_df = pd.DataFrame(curr["currencies"].items(), columns=["Target_Currency", "Description"])

  #merging the currecny dataframe with the current df based on Target_Currency
  df = df.merge(currency_df, on="Target_Currency", how="left")
  df['Created_at'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

  #Format the data types for each column
  df = df.astype({
      "Date" : "datetime64[ns]",
      "Target_Currency" : "string",
      "Exchange_Rate" : "float64",
      "Rate_in_IDR" : "float64",
      "Base_Currency" : "string",
      "Description" : "string",
      "Created_at" : "datetime64[ns]"
      })

  #Reorder the columns order
  column_order = ['Date','Target_Currency','Description','Exchange_Rate','Rate_in_IDR','Base_Currency','Created_at']
  df = df[column_order]

  return df

In [8]:
idr_rate = extract_curr('IDR')

In [9]:
df = transform_curr(idr_rate)

df.head()

,Date,Target_Currency,Description,Exchange_Rate,Rate_in_IDR,Base_Currency,Created_at
0,2025-09-01,AED,United Arab Emirates Dirham,0.000223,4484.304933,IDR,2025-10-26 18:36:01
1,2025-09-01,AFN,Afghan Afghani,0.004162,240.269101,IDR,2025-10-26 18:36:01
2,2025-09-01,ALL,Albanian Lek,0.005083,196.734212,IDR,2025-10-26 18:36:01
3,2025-09-01,AMD,Armenian Dram,0.023276,42.962708,IDR,2025-10-26 18:36:01
4,2025-09-01,ANG,Netherlands Antillean Guilder,0.000109,9174.311927,IDR,2025-10-26 18:36:01


In [10]:
print(df.dtypes)

Date               datetime64[ns]
Target_Currency    string[python]
Description        string[python]
Exchange_Rate             float64
Rate_in_IDR               float64
Base_Currency      string[python]
Created_at         datetime64[ns]
dtype: object


In [11]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9234 entries, 0 to 9233
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Date             9234 non-null   datetime64[ns]
 1   Target_Currency  9234 non-null   string        
 2   Description      9234 non-null   string        
 3   Exchange_Rate    9234 non-null   float64       
 4   Rate_in_IDR      9234 non-null   float64       
 5   Base_Currency    9234 non-null   string        
 6   Created_at       9234 non-null   datetime64[ns]
dtypes: datetime64[ns](2), float64(2), string(3)
memory usage: 505.1 KB
None


In [12]:
def load_to_bigquery(df,table_name):
    client = bigquery.Client()
    
    #Define the name of the project, dataset, and table
    project_id = "exchange-rate-pipeline-476115"
    dataset_id = "currency_data"
    table_id = table_name
    
    table_ref = f"{project_id}.{dataset_id}.{table_id}"
    try: 
        job_config = bigquery.LoadJobConfig(
            write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE,
        )

        #Load the dataframe to bigquery's table
        job = client.load_table_from_dataframe(df, table_ref, job_config = job_config)
        job.result()

        table = client.get_table(table_ref)
        print("Data berhasil dimuat")
    
    except Exception as e:
        print("Data gagal dimuat",e)




In [13]:
table_exchange_rate = "exchange_rate"
df = load_to_bigquery(df,table_exchange_rate)

d:\Python\Financial_data_pipeline\venv\Lib\site-packages\google\cloud\bigquery\_pandas_helpers.py:484: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


Data berhasil dimuat


In [14]:
# def load_to_mysql(df):
#   #connect the MySQL database
#   usr = 'root'
#   pwd = 'root'
#   host = '127.0.0.1'
#   port = 3306
#   dbName = 'currency'
#   tableName = 'exchange_rate'
#   try:
#     engine = create_engine(f"mysql+mysqldb://{usr}:{pwd}@{host}:{port}/{dbName}",
#                            echo=True,
#                            future=True)
#     #Send the dataframes to MySQL table
#     df.to_sql(name=tableName, con=engine, if_exists="replace", index=False)

#     print("Data berhasil dimuat ke tabel 'exchange_rate' di MySQL")
#   except Exception as e:
#     print("Gagal memuat data ke mysql:",e)

In [15]:
# df = load_to_mysql(df)

In [16]:
# #Filtering the dataframes by Target_Currency with the values is JPY
# df[df['Target_Currency'] == 'JPY']